<a href="https://colab.research.google.com/github/rubymanderna/ML_ECGR5105/blob/main/Homework_5/Homework_5_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 2 (40 pts):

2.a. Develop preprocessing and a training loop to train a linear regression model that predicts housing price based on the following input variables:

area, bedrooms, bathrooms, stories, parking

For this, you need to use the housing dataset. For training and validation, use 80% (training) and 20% (validation) split. Identify the best parameters for your linear regression model based on the above input variables. In this case, you will have six parameters:

2.b Use 5000 epochs for your training. Explore different learning rates from 0.1 to 0.0001 (you need four separate trainings). Report your loss and validation accuracy for every 500 epochs per training. Pick the best linear model.

2.c. Compare your results against the linear regression done in homework 1. Do you see meaningful differences?

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDatasetfrom sklearn.linear_model import LinearRegression

df = pd.read_csv('/content/drive/MyDrive/ML_ECGR5105/Housing.csv')

In [6]:
df.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [9]:


# Select relevant features
selected_features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']
df = df[selected_features]

# Train-Validation Split
X = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
y = df['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the Input Features
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader for training set
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define a simple linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

# Linear Regression Model
input_size = X_train.shape[1]
model = LinearRegressionModel(input_size)

# Training Loop with SGD optimizer and tracking best model
n_epochs = 5000
learning_rates = [0.1, 0.01, 0.001, 0.0001]

criterion = nn.MSELoss()
best_model_sgd = None
best_mse_sgd = float('inf')

print("Training with SGD optimizer:")
for lr in learning_rates:
    print(f"Learning rate: {lr}")
    optimizer_sgd = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for batch_X, batch_y in train_loader:
            # Forward pass
            y_pred = model(batch_X)

            # Compute the loss
            loss = criterion(y_pred, batch_y)

            # Backward pass and optimization
            optimizer_sgd.zero_grad()
            loss.backward()
            optimizer_sgd.step()

        if epoch % 500 == 0:
            # Validation
            y_pred_val = model(X_val_tensor)
            mse = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val.detach().numpy())
            print(f'Epoch {epoch}, Mean Squared Error (SGD): {mse}')

            # Track the best model based on validation loss for SGD
            if mse < best_mse_sgd:
                best_mse_sgd = mse
                best_model_sgd = model.state_dict()

# Load the best SGD model
model.load_state_dict(best_model_sgd)

# Test the best SGD model on the validation set
y_pred_val_sgd = model(X_val_tensor)
mse_sgd = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val_sgd.detach().numpy())
print(f'\nBest Model (SGD) - Mean Squared Error on Validation Set: {mse_sgd}')


# Training Loop with Adam optimizer and tracking best model
best_model_adam = None
best_mse_adam = float('inf')

print("\nTraining with Adam optimizer:")
for lr in learning_rates:
    print(f"Learning rate: {lr}")
    optimizer_adam = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for batch_X, batch_y in train_loader:
            # Forward pass
            y_pred = model(batch_X)

            # Compute the loss
            loss = criterion(y_pred, batch_y)

            # Backward pass and optimization
            optimizer_adam.zero_grad()
            loss.backward()
            optimizer_adam.step()

        if epoch % 500 == 0:
            # Validation
            y_pred_val = model(X_val_tensor)
            mse = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val.detach().numpy())
            print(f'Epoch {epoch}, Mean Squared Error (Adam): {mse}')

            # Track the best model based on validation loss for Adam
            if mse < best_mse_adam:
                best_mse_adam = mse
                best_model_adam = model.state_dict()

# Load the best Adam model
model.load_state_dict(best_model_adam)

# Test the best Adam model on the validation set
y_pred_val_adam = model(X_val_tensor)
mse_adam = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val_adam.detach().numpy())
print(f'\nBest Model (Adam) - Mean Squared Error on Validation Set: {mse_adam}')


Training with SGD optimizer:
Learning rate: 0.1
Epoch 0, Mean Squared Error (SGD): 2494470815744.0
Epoch 500, Mean Squared Error (SGD): 2359427334144.0
Epoch 1000, Mean Squared Error (SGD): 2330197491712.0
Epoch 1500, Mean Squared Error (SGD): 2265070174208.0
Epoch 2000, Mean Squared Error (SGD): 2254688223232.0
Epoch 2500, Mean Squared Error (SGD): 2173691756544.0
Epoch 3000, Mean Squared Error (SGD): 2311604666368.0
Epoch 3500, Mean Squared Error (SGD): 2301735993344.0
Epoch 4000, Mean Squared Error (SGD): 2234002964480.0
Epoch 4500, Mean Squared Error (SGD): 2350491893760.0
Learning rate: 0.01
Epoch 0, Mean Squared Error (SGD): 2340860723200.0
Epoch 500, Mean Squared Error (SGD): 2301604921344.0
Epoch 1000, Mean Squared Error (SGD): 2300179906560.0
Epoch 1500, Mean Squared Error (SGD): 2303374131200.0
Epoch 2000, Mean Squared Error (SGD): 2289769119744.0
Epoch 2500, Mean Squared Error (SGD): 2295129702400.0
Epoch 3000, Mean Squared Error (SGD): 2291934429184.0
Epoch 3500, Mean Squar

#**Best model**

Best Model (SGD) - Mean Squared Error on Validation Set: 2292684947456.0

Best Model (Adam) - Mean Squared Error on Validation Set: 2292676820992.0


*2*.c. Compare your results against the linear regression done in homework 1. Do you see meaningful differences?


homework 2 result

**For input normalization results for 3b Added Penality**

Best Learning Rate: 0.01
Best MSE: 900238626517.4305

**For input normalization results for 2b without Penality**

Best Learning Rate: 0.1
Best MSE: 894200375157.158


After comparing with homework 2 model with ADAM optimizer is performing better as it has min loss

Problem 3 (30 pts):

Repeat all sections of problem 2 using all the input features from the housing price dataset.

In [10]:


X = df.drop('price', axis=1)
y = df['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the Input Features
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader for training set
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define a simple linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

# Linear Regression Model
input_size = X_train.shape[1]
model = LinearRegressionModel(input_size)

# Training Loop with SGD optimizer and tracking best model
n_epochs = 5000
learning_rates = [0.1, 0.01, 0.001, 0.0001]

criterion = nn.MSELoss()
best_model_sgd = None
best_mse_sgd = float('inf')

print("Training with SGD optimizer:")
for lr in learning_rates:
    print(f"Learning rate: {lr}")
    optimizer_sgd = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for batch_X, batch_y in train_loader:
            # Forward pass
            y_pred = model(batch_X)

            # Compute the loss
            loss = criterion(y_pred, batch_y)

            # Backward pass and optimization
            optimizer_sgd.zero_grad()
            loss.backward()
            optimizer_sgd.step()

        if epoch % 500 == 0:
            # Validation
            y_pred_val = model(X_val_tensor)
            mse = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val.detach().numpy())
            print(f'Epoch {epoch}, Mean Squared Error (SGD): {mse}')

            # Track the best model based on validation loss for SGD
            if mse < best_mse_sgd:
                best_mse_sgd = mse
                best_model_sgd = model.state_dict()

# Load the best SGD model
model.load_state_dict(best_model_sgd)

# Test the best SGD model on the validation set
y_pred_val_sgd = model(X_val_tensor)
mse_sgd = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val_sgd.detach().numpy())
print(f'\nBest Model (SGD) - Mean Squared Error on Validation Set: {mse_sgd}')


# Training Loop with Adam optimizer and tracking best model
best_model_adam = None
best_mse_adam = float('inf')

print("\nTraining with Adam optimizer:")
for lr in learning_rates:
    print(f"Learning rate: {lr}")
    optimizer_adam = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for batch_X, batch_y in train_loader:
            # Forward pass
            y_pred = model(batch_X)

            # Compute the loss
            loss = criterion(y_pred, batch_y)

            # Backward pass and optimization
            optimizer_adam.zero_grad()
            loss.backward()
            optimizer_adam.step()

        if epoch % 500 == 0:
            # Validation
            y_pred_val = model(X_val_tensor)
            mse = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val.detach().numpy())
            print(f'Epoch {epoch}, Mean Squared Error (Adam): {mse}')

            # Track the best model based on validation loss for Adam
            if mse < best_mse_adam:
                best_mse_adam = mse
                best_model_adam = model.state_dict()

# Load the best Adam model
model.load_state_dict(best_model_adam)

# Test the best Adam model on the validation set
y_pred_val_adam = model(X_val_tensor)
mse_adam = mean_squared_error(y_val_tensor.detach().numpy(), y_pred_val_adam.detach().numpy())
print(f'\nBest Model (Adam) - Mean Squared Error on Validation Set: {mse_adam}')


Training with SGD optimizer:
Learning rate: 0.1
Epoch 0, Mean Squared Error (SGD): 2436422696960.0
Epoch 500, Mean Squared Error (SGD): 2426676969472.0
Epoch 1000, Mean Squared Error (SGD): 2335065767936.0
Epoch 1500, Mean Squared Error (SGD): 2207368347648.0
Epoch 2000, Mean Squared Error (SGD): 2459445493760.0
Epoch 2500, Mean Squared Error (SGD): 2501334007808.0
Epoch 3000, Mean Squared Error (SGD): 2195636617216.0
Epoch 3500, Mean Squared Error (SGD): 2225410408448.0
Epoch 4000, Mean Squared Error (SGD): 2304953286656.0
Epoch 4500, Mean Squared Error (SGD): 2290366021632.0
Learning rate: 0.01
Epoch 0, Mean Squared Error (SGD): 2244973690880.0
Epoch 500, Mean Squared Error (SGD): 2291586301952.0
Epoch 1000, Mean Squared Error (SGD): 2293496807424.0
Epoch 1500, Mean Squared Error (SGD): 2302454267904.0
Epoch 2000, Mean Squared Error (SGD): 2302360944640.0
Epoch 2500, Mean Squared Error (SGD): 2297196183552.0
Epoch 3000, Mean Squared Error (SGD): 2285594738688.0
Epoch 3500, Mean Squar

# BEST MODEL



Best Model (SGD) - Mean Squared Error on Validation Set: 2292845641728.0

Best Model (Adam) - Mean Squared Error on Validation Set: 2292854554624.0

Best model with ADAM optimizer performed better

homework 2 result

**For input normalization results for 3b Added Penality**

Best Learning Rate: 0.01
Best MSE: 900238626517.4305

**For input normalization results for 2b without Penality**

Best Learning Rate: 0.1
Best MSE: 894200375157.158

After comparing the results model with the ADAM optimizer is performing better and as min loss